In [22]:
import pandas as pd
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm

xml_file_path = '../data/Task2_3_combined/NBER_working_papers.xml'
tree = ET.parse(xml_file_path)
root = tree.getroot()

title = root.find('.//atl').text #find returns first instance of title, abstract
abstract = root.find('.//ab').text
data = []
for rec in root.findall('rec'):
    header = rec.find('header')
    #result_id = rec.get('resultID')
    
    #short_db_name = header.get('shortDbName')
    #long_db_name = header.get('longDbName')
    #ui_term = header.get('uiTerm')

    control_info = header.find('controlInfo')
    pubinfo = control_info.find('pubinfo')
    artinfo = control_info.find('artinfo')
    display_info = header.find('displayInfo')
    
    article_data = {
        'publication_year': pubinfo.find('dt').text if pubinfo.find('dt') is not None else None,
        'jtl': control_info.find('./jinfo/jtl').text if control_info.find('./jinfo/jtl') is not None else None,
        'doi': artinfo.find('ui').text if artinfo.find('ui') is not None else None,
        'title': artinfo.find('./tig/atl').text if artinfo.find('./tig/atl') is not None else None,
        'authors': ', '.join([au.text for au in artinfo.findall('./aug/au')]) if artinfo.findall('./aug/au') else None,
        'abstract': artinfo.find('ab').text if artinfo.find('ab') is not None else None,
        'url': display_info.find('./pLink/url').text if display_info.find('./pLink/url') is not None else None
    }
    
    data.append(article_data)

df = pd.DataFrame(data)
df['GPT-Evaluation'] = -1


In [23]:
df

,publication_year,jtl,doi,title,authors,abstract,url,GPT-Evaluation
0,20240101,None,None,A Supply Curve for Forest-Based CO2 Removal,"Franklin, Sergio L., Jr., Pindyck, Robert S.",Forestation is viewed as an important means of...,https://search.ebscohost.com/login.aspx?direct...,-1
1,20240101,None,None,New Area- and Population-based Geographic Cros...,"Ferrara, Andreas, Testa, Patrick A., Zhou, Liyang","In applied historical research, geographic uni...",https://search.ebscohost.com/login.aspx?direct...,-1
2,20240101,None,None,Decarbonizing Aviation: Cash-for-Clunkers in t...,"Brueckner, Jan K., Kahn, Matthew E., Nickelsbu...",The durability of the transportation capital s...,https://search.ebscohost.com/login.aspx?direct...,-1
3,20240101,None,None,The Multigenerational Impact of Children and C...,"Karademir, Sencer, Laliberte, Jean-William P.,...",This paper examines the multigenerational impa...,https://search.ebscohost.com/login.aspx?direct...,-1
4,20240101,None,None,Inequality Within Countries is Falling: Underr...,"Pinkovskiy, Maxim, Sala-i-Martin, Xavier, Chat...",Household surveys suffer from persistent and g...,https://search.ebscohost.com/login.aspx?direct...,-1
...,...,...,...,...,...,...,...,...
2576,20220101,None,None,Global Innovation and Knowledge Diffusion,"Lind, Nelson, Ramondo, Natalia",We develop a Ricardian model of trade in which...,https://search.ebscohost.com/login.aspx?direct...,-1
2577,20220101,None,None,"Macroeconomic Research, Present and Past","Glandon, Philip J., Kuttner, Kenneth, Mazumder...",How is macroeconomic research conducted and wh...,https://search.ebscohost.com/login.aspx?direct...,-1
2578,20220101,None,None,Revealing Corruption: Firm and Worker Level Ev...,"Colonnelli, Emanuele, Lagaras, Spyridon, Ponti...",We study how the disclosure of corrupt practic...,https://search.ebscohost.com/login.aspx?direct...,-1
2579,20220101,None,None,Debt as Safe Asset,"Brunnermeier, Markus K., Merkel, Sebastian A.,...",The price of a safe asset reflects not only th...,https://search.ebscohost.com/login.aspx?direct...,-1


In [24]:
jsonl_path = '../data/Task2_3_combined/NBER_workingpapers_batch-output.jsonl'

with open(jsonl_path, 'r') as outputs:
    for (index, row), output_line in zip(df.iterrows(), outputs):
        json_output_obj = json.loads(output_line)
        
        gpt_eval = json_output_obj.get('response', {}).get('body', {}).get('choices', {})[0].get('message').get('content')
        df.at[index, 'GPT-Evaluation'] = gpt_eval
        df.at[index, 'publication_year'] = df.at[index, 'publication_year'][:4]

df.to_excel('../data/Task2_3_combined/NBER_workingpapers_of_interest.xlsx', index=False)

C:\Users\ramkr\AppData\Local\Temp\ipykernel_23688\3474137303.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, 'GPT-Evaluation'] = gpt_eval


In [37]:
count_tens = (df["GPT-Evaluation"] == "9").sum()
print("Number of entries equal to '10':", count_tens)


Number of entries equal to '10': 6


In [30]:
count = (df["GPT-Evaluation"] == "6").sum()
print(count)


6


##### Combine two excel sheets

In [39]:
import pandas as pd

df1 = pd.read_excel('../data/Task2_3_combined/source_papers_of_interest.xlsx', sheet_name='Sheet1')
df2 = pd.read_excel('../data/Task2_3_combined/NBER_workingpapers_of_interest.xlsx', sheet_name='Sheet1')

output_file = '../data/Task2_3_combined/result.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df1.to_excel(writer, sheet_name='Sheet1', index=False)
    df2.to_excel(writer, sheet_name='Sheet2', index=False)

print("Sheets combined into:", output_file)

Sheets combined into: ../data/Task2_3_combined/result.xlsx
